# Weekly GitHub Activity Fetcher

This notebook fetches your GitHub activity (commits, pull requests, issues) from the last 7 days, as described in the project requirements.

In [48]:
# Set Up Environment Variables
import os
from dotenv import load_dotenv

# Load environment variables from .env file if present
load_dotenv()

PERSONAL_GITHUB_TOKEN = os.getenv('PERSONAL_GITHUB_TOKEN')
PERSONAL_GITHUB_USERNAME = os.getenv('PERSONAL_GITHUB_USERNAME')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
NOTION_TOKEN = os.getenv('NOTION_TOKEN')
NOTION_DATABASE_ID = os.getenv('NOTION_DATABASE_ID')

if not PERSONAL_GITHUB_TOKEN:
    raise ValueError("PERSONAL_GITHUB_TOKEN must be set in environment variables.")

if not PERSONAL_GITHUB_USERNAME:
    raise ValueError("PERSONAL_GITHUB_USERNAME must be set in environment variables.")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY must be set in environment variables.")

if not NOTION_TOKEN:
    raise ValueError("NOTION_TOKEN must be set in environment variables.")

if not NOTION_DATABASE_ID:
    raise ValueError("NOTION_DATABASE_ID must be set in environment variables.")

In [49]:
# Import Required Libraries
import requests
from datetime import datetime, timedelta

In [ ]:
# Define Function to Fetch GitHub Activity

def fetch_github_activity(username, token):
    """
    Fetch commits (push events), pull requests, and issues from the last 7 days for the given user.
    Group by repository, event type, and description.
    """
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    since = (datetime.now() - timedelta(days=7)).isoformat() + "Z"
    events_url = f"https://api.github.com/users/{username}/events"
    response = requests.get(events_url, headers=headers)
    response.raise_for_status()
    events = response.json()
    
    events = [event for event in events if event.get("type", "") == "PushEvent" or event.get("type", "") == "PullRequestEvent"]
    
    activity = []
    for event in events:
        created_at = event.get("created_at", "")
        if created_at < since:
            continue
        repo = event["repo"]["name"]
        type_ = event["type"]
        desc = ""
        body = ""
        if type_ == "PushEvent":
            desc = f"Pushed {len(event['payload']['commits'])} commit(s)"
            body = "\n".join(commit["message"] for commit in event["payload"]["commits"])
        elif type_ == "PullRequestEvent":
            pr = event["payload"]["pull_request"]
            desc = f"PR: {pr['title']} (#{pr['number']})"
            body = pr["body"]
        else:
            continue
        activity.append({
            "repo": repo,
            "type": type_,
            "desc": desc,
            "created_at": created_at,
            "body": body
        })
    return activity

In [51]:
# Fetch and Display GitHub Activity
activity = fetch_github_activity(PERSONAL_GITHUB_USERNAME, PERSONAL_GITHUB_TOKEN)

if activity:
    print("GitHub Activity (last 7 days):\n")
    for item in activity:
        print(f"[{item['created_at']}] {item['repo']} - {item['type']}: {item['desc']} {item['body']}")
else:
    print("No activity found in the last 7 days.")

GitHub Activity (last 7 days):

[2025-07-16T17:45:41Z] ravgeetdhillon/ravgeet-web - PushEvent: Pushed 1 commit(s) chore(ci): simplify production deploy workflow

- Remove Netlify deployment steps
- Switch to Vercel deploy hook only
- Update cron schedule for more frequent deploys
[2025-07-16T13:35:55Z] ravgeetdhillon/blog-publishing - PushEvent: Pushed 1 commit(s) fix: handle nonetype values
[2025-07-16T13:09:43Z] ravgeetdhillon/blog-publishing - PushEvent: Pushed 1 commit(s) fix: better null handling
[2025-07-16T10:12:11Z] cloudanswers/designeradvantage-2 - PullRequestEvent: PR: fix: fix typo in variable names (#2749) <!-- short summary of the changes introduced -->
<!-- What kind of change does this PR introduce?  (new feature, bugfix, code improvement etc.) -->

### Related Issues

<!-- link to related issues -->

### Screenshot or Video of Change

<!-- paste a screenshot or attach/link a video for ui changes so more people can give input and to make QA and review faster -->

### No

## Summarize Weekly GitHub Activity

Aggregate the fetched GitHub activity into a readable summary for AI content generation.

In [52]:
# Summarize Weekly GitHub Activity

def summarize_activity(activity):
    """
    Create a readable summary from the activity list for AI input.
    """
    if not activity:
        return "No activity found in the last 7 days."
    summary_lines = ["Last week:"]
    for item in activity:
        if item['type'] == 'PushEvent':
            summary_lines.append(f"- {item['desc']} in {item['repo']} with content: {item['body']}")
        elif item['type'] == 'PullRequestEvent':
            summary_lines.append(f"- Merged {item['desc']} in {item['repo']} with content: {item['body']}")
    return "\n".join(summary_lines)

weekly_summary = summarize_activity(activity)
print(len(weekly_summary))
print(weekly_summary)

18031
Last week:
- Pushed 1 commit(s) in ravgeetdhillon/ravgeet-web with content: chore(ci): simplify production deploy workflow

- Remove Netlify deployment steps
- Switch to Vercel deploy hook only
- Update cron schedule for more frequent deploys
- Pushed 1 commit(s) in ravgeetdhillon/blog-publishing with content: fix: handle nonetype values
- Pushed 1 commit(s) in ravgeetdhillon/blog-publishing with content: fix: better null handling
- Merged PR: fix: fix typo in variable names (#2749) in cloudanswers/designeradvantage-2 with content: <!-- short summary of the changes introduced -->
<!-- What kind of change does this PR introduce?  (new feature, bugfix, code improvement etc.) -->

### Related Issues

<!-- link to related issues -->

### Screenshot or Video of Change

<!-- paste a screenshot or attach/link a video for ui changes so more people can give input and to make QA and review faster -->

### Notes for QA

<!-- a short summary of anything to test or which doesn't need to be te

## Generate Marketing Content with AI

Use an AI model (e.g., OpenAI GPT) to generate 5 unique marketing posts for Mon–Fri based on the weekly summary.

In [63]:
# Generate Marketing Content with AI
import openai
import json

openai.api_key = OPENAI_API_KEY

def generate_linkedin_post_ideas(summary):
    prompt = f"""
You are a social media marketer.

Task:
Based on the following weekly GitHub activity summary, list 5 unique topics that I worked on that can be later turned into LinkedIn posts.

Note:
- Donot add any emojis
- Provide the response in a JSON format with the following structure with no markdown. Donot wrap the JSON response in any other text or markdown or code blocks:
- heading: A short title for the post
- body: A detailed explanation of the topic suitable for a LinkedIn post

Summary:
{summary}
"""
    response = openai.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

if openai.api_key:
    marketing_posts = generate_linkedin_post_ideas(weekly_summary)
    print(len(marketing_posts))
    print(marketing_posts)
    marketing_posts = json.loads(marketing_posts)
else:
    print("OPENAI_API_KEY not set. Please add your OpenAI API key to the environment.")

3936
[
  {
    "heading": "Streamlining Production Deployment Workflows with Vercel",
    "body": "This week, I focused on optimizing the deployment process for a web application by simplifying the continuous integration pipeline. The deployment was transitioned from a mixed workflow with Netlify to a fully streamlined approach leveraging Vercel deploy hooks. By removing unnecessary steps and updating the deploy cron schedule for increased frequency, we achieved a more reliable and faster path to production. This change minimizes operational friction, improves developer experience, and ensures our releases are both more predictable and maintainable. I plan to share the lessons learned and best practices for those considering a similar migration in their production workflows."
  },
  {
    "heading": "Mastering Null and None Handling in Python Data Processing",
    "body": "Robust null handling is essential for any data-driven application. Last week, I dedicated effort to enhancing the 

In [60]:
brief = marketing_posts[0]["body"]
brief

'This week, I streamlined our CI/CD workflows by switching production deployments from Netlify to the much sleeker Vercel deploy hooks. Along with removing outdated Netlify steps, I updated our cron schedules for more frequent deploys — because who doesn’t love things shipping faster? 🚀 Continuous improvement, continuous deployment!'

In [ ]:
# Generate Marketing Content with AI
import openai
import json

openai.api_key = OPENAI_API_KEY

def generate_linkedin_post(brief):
    prompt = f"""
You are a social media marketer and a good story writer.

Your tasks:
Based on the following brief, create a LinkedIn post.
Make it personal and add story-telling. 
Also, add reasons around why would I have done this.

Notes:
- Always write in first person and write like a human not like a bot
- Be free to add emojis and very tiny bit of humor if relevant.
- Only provide the post body without any markdown.
- Donot wrap the JSON response in any other text or markdown or code blocks.
- Keep the lanaguage simple, professional yet engaging.

Brief:
{brief}
"""
    response = openai.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

if openai.api_key:
    post = generate_linkedin_post(brief)
    print(len(post))
    print(post)
else:
    print("OPENAI_API_KEY not set. Please add your OpenAI API key to the environment.")

951
This week, I tackled something that had been slowing us down for a while. Our CI/CD pipeline felt clunky, with old Netlify steps lingering and deploys not quite as snappy as I knew we deserved.

So I rolled up my sleeves and migrated our production deployments from Netlify to Vercel deploy hooks. The difference was immediate—sleeker workflows, cleaner integration, and the flexibility to really fine-tune how and when our updates go live.

But I didn’t stop there. I re-examined our cron schedules and made deploys more frequent. Why? Because I believe in getting better not just for the sake of improvement, but so our team can deliver value faster and more often. There’s nothing quite like the feeling of shipping improvements at a pace that matches our ambition.

Continuous improvement and continuous deployment may sound like buzzwords, but for me, it’s about making everyone’s day a bit easier—and our product a lot better. Onward and upward!


## Save Generated Posts to Notion Database

Use the Notion API to insert each generated post into your Notion database for review and scheduling.

In [ ]:
# Save Generated Posts to Notion Database
import requests
import json


headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

def add_post_to_notion(post, date):
    url = f"https://api.notion.com/v1/pages"
    data = {
        "parent": {"database_id": NOTION_DATABASE_ID},
        "properties": {
            "Title": {"title": [{"text": {"content": post["heading"]}}]},
            "Due Date": {"date": {"start": date}},
            "Status": {"select": {"name": "To Do"}},
            "Type": {"select": {"name": "Marketing"}},
        },
        "children": [
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [{"type": "text","text": {"content": post["body"]}}]
                }
            }
        ]
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        print(f"Post {post["heading"]} added to Notion for {date}.")
    else:
        print(f"Failed to add post for {date}: {response.text}")



# Example usage: assuming marketing_posts is a list of dicts with 'body' and 'heading' keys
from datetime import datetime, timedelta
if NOTION_TOKEN and NOTION_DATABASE_ID:
    today = datetime.today()
    for i, post in enumerate(marketing_posts):
        post_date = (today + timedelta(days=i)).strftime('%Y-%m-%d')
        add_post_to_notion(post, post_date)
else:
    print("NOTION_TOKEN or NOTION_DATABASE_ID not set. Please add them to your environment.")